# Healthy Café Location Selection in Singapore

IMB Data Science Capstone Project – The Battles of Neighbourhoods

<b>Introduction<br>
I.	Background</b>

About 400,000 Singaporeans are diabetes and one in three has lifetime risk of developing the disease. And if nothing is done, the number of diabetes under age 70 is expected to rise to 670,000 by 2030 and one million by 2050.
<br><br>
Singapore government adopts a multi-pronged strategy to encourage Singaporeans to eat healthily and reduce their sugar intake from foods and drinks.<br>
Coffee is one of the common sugar intakes for Singaporean. We are going to open a café to server coffee with less sugar and food with balanced nutrition.


<b>II.	Healthy Café Concept</b>

<b>Target customers:</b> Office clericals who care about healthy lifestyle but don’t have time to prepare food for lunch or rush for dinner due to overtime.<br>
<b>Food Service:</b> Coffee with sugar level choices and different types of milk (e.g. non-fat, skimmed milk, soya milk, etc). Food with nutrition label, and different size options to match and mix.


<b>III.	Objective</b><br>
To find out suitable locations in Singapore<br>
a.	Near business area<br>
b.	Ares with high density of gyms (To filter those areas passed by more people with healthy concept)<br>
c.	Distribution of restaurants (types and density) – to understand competitor distribution<br>

<b>Data Acquisition</b><br>
I.	Target business area in Singapore: https://www.corporateservicessingapore.com/7-popular-business-locations-singapore/ <br>
So, we can get below starting points<br>
1.	Raffles Place Area<br>
2.	Marina Bay Area<br>
3.	Tanjong Pagar / Anson Road<br>
4.	Orchard Road Area<br>
5.	Shenton Way Area<br>
6.	River Valley<br>
7.	Suntec City<br>
<br>

Uploaded datafile to GitHub as a start.

In [4]:
!conda install -c conda-forge folium=0.5.0 --yes # comment/uncomment if not yet installed.
!conda install -c conda-forge geopy --yes        # comment/uncomment if not yet installed

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

# Numpy and Pandas libraries were already imported at the beginning of this notebook.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

import requests # library to handle requests
import lxml.html as lh
import bs4 as bs
import urllib.request

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.3.0               |           py36_0         747 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [5]:
data_area=pd.read_csv("https://raw.githubusercontent.com/shayy07/Coursera_Capstone/master/OfficeArea.csv")
data_area

,Area,Landmark
0,Raffles Place Area,Raffles Place MRT
1,Marina Bay Area,Marina Bay Finacial Centre
2,Tanjong Pagar / Anson Road,Tanjong Pagar MRT
3,Orchard Road Area,Ngee Ann City
4,Shenton Way Area,Capital Tower
5,River Valley,Great World City
6,Suntec City,Suntec City


In [6]:
MyLM=data_area["Landmark"]
MyLM

0             Raffles Place MRT
1    Marina Bay Finacial Centre
2             Tanjong Pagar MRT
3                 Ngee Ann City
4                 Capital Tower
5              Great World City
6                   Suntec City
Name: Landmark, dtype: object

II.	Get geocodes of above areas via Google Map

In [8]:
data_area['Latitude'] = 0.0
data_area['Longitude'] = 0.0

for idx,area in data_area['Landmark'].iteritems():
    area=area + ' ' + 'Singapore'
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}'.format(area,google_key)
    #url='https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={}&key={}.format(area,google_key)
    lat = requests.get(url).json()["results"][0]["geometry"]["location"]['lat']
    lng = requests.get(url).json()["results"][0]["geometry"]["location"]['lng']
    data_area.loc[idx,'Latitude'] = lat
    data_area.loc[idx,'Longitude'] = lng
    
data_area

,Area,Landmark,Latitude,Longitude
0,Raffles Place Area,Raffles Place MRT,1.283969,103.851540
1,Marina Bay Area,Marina Bay Finacial Centre,1.280283,103.854307
2,Tanjong Pagar / Anson Road,Tanjong Pagar MRT,1.276342,103.846792
3,Orchard Road Area,Ngee Ann City,1.302557,103.834568
4,Shenton Way Area,Capital Tower,1.277737,103.847622
5,River Valley,Great World City,1.293642,103.831929
6,Suntec City,Suntec City,1.294780,103.858526


III.	Collect location data from Foursquare website: https://developer.foursquare.com/<br>
1.	Location of Venues

In [89]:
url = 'https://api.foursquare.com/v2/venues/explore'
venue_list=[]

In [90]:
for idx,lat in data_area['Latitude'].iteritems():
    lng=data_area.loc[idx,'Longitude']
    name=data_area.loc[idx,'Area']
    sll=str(lat) + ',' + str(lng)
    params = dict(
      client_id=cid,
      client_secret=csecret,
      v='20180323',
      ll=sll,
      radius=300,
      query='food'
    )
    resp = requests.get(url=url, params=params).json()["response"]['groups'][0]['items']
    main_category="Other Food"
    venue_list.append([(name,lat,lng,main_category,v['venue']['id'],v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name']) for v in resp])

In [91]:
for idx,lat in data_area['Latitude'].iteritems():
    lng=data_area.loc[idx,'Longitude']
    name=data_area.loc[idx,'Area']
    sll=str(lat) + ',' + str(lng)
    params = dict(
      client_id=cid,
      client_secret=csecret,
      v='20180323',
      ll=sll,
      radius=300,
      query='Café'
    )
    resp = requests.get(url=url, params=params).json()["response"]['groups'][0]['items']
    main_category="Café"
    venue_list.append([(name,lat,lng,main_category,v['venue']['id'],v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name']) for v in resp])

In [92]:
for idx,lat in data_area['Latitude'].iteritems():
    lng=data_area.loc[idx,'Longitude']
    name=data_area.loc[idx,'Area']
    sll=str(lat) + ',' + str(lng)
    params = dict(
      client_id=cid,
      client_secret=csecret,
      v='20180323',
      ll=sll,
      radius=300,
      query='gym'
    )
    resp = requests.get(url=url, params=params).json()["response"]['groups'][0]['items']
    main_category="Gym"
    venue_list.append([(name,lat,lng,main_category,v['venue']['id'],v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name']) for v in resp])

In [93]:
nearby_venues = pd.DataFrame([item for venue_list in venue_list for item in venue_list])
nearby_venues.columns = ['Area','Area_Latitude','Area_Longitude','Venue_Main_Category','Venue_ID','Venue','Venue_Latitude','Venue_Longitude','Venue_Category']

nearby_venues.head()

,Area,Area_Latitude,Area_Longitude,Venue_Main_Category,Venue_ID,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Raffles Place Area,1.283969,103.85154,Other Food,53ae48b4498ec970f3cc0455,CITY Hot Pot Shabu shabu,1.284173,103.851585,Hotpot Restaurant
1,Raffles Place Area,1.283969,103.85154,Other Food,55fe3da6498e0c5eaa442250,CULINARYON,1.284876,103.850933,Comfort Food Restaurant
2,Raffles Place Area,1.283969,103.85154,Other Food,59538b0ac824ae5235423946,Waa Cow!,1.284284,103.851215,Japanese Restaurant
3,Raffles Place Area,1.283969,103.85154,Other Food,4bb2edcb2397b713669e37b3,The Salad Shop,1.285523,103.851177,Salad Place
4,Raffles Place Area,1.283969,103.85154,Other Food,4b568e7cf964a520151528e3,The Sandwich Shop,1.284266,103.852673,Sandwich Place


In [95]:
nearby_venues.groupby(['Venue_Main_Category'])['Venue_Category'].value_counts(normalize=False)

Venue_Main_Category  Venue_Category                          
Café                 Café                                        173
                     Coffee Shop                                   4
                     Bar                                           2
                     Fast Food Restaurant                          1
Gym                  Gym / Fitness Center                         35
                     Gym                                          27
                     Yoga Studio                                  13
                     Hotel                                         3
                     Residential Building (Apartment / Condo)      3
                     Boxing Gym                                    2
                     Climbing Gym                                  2
                     Cycle Studio                                  2
                     Gym Pool                                      2
                     Martial Arts Dojo   

2. Clean data

In [118]:
nearby_venues_clean=nearby_venues.copy()

list_cafe_filter=['Bar','Fast Food Restaurant']
indexNames = nearby_venues_clean[ (nearby_venues_clean['Venue_Main_Category'] == 'Café') & (nearby_venues_clean['Venue_Category'].isin(list_cafe_filter)) ].index
nearby_venues_clean.drop(indexNames , inplace=True)

list_Gym_filter=['Hotel','Residential Building (Apartment / Condo)','Martial Arts Dojo','Building','Hotel Pool','Track']
indexNames = nearby_venues_clean[ (nearby_venues_clean['Venue_Main_Category'] == 'Gym') & (nearby_venues_clean['Venue_Category'].isin(list_Gym_filter)) ].index
nearby_venues_clean.drop(indexNames , inplace=True)

list_Food_filter=['Café']
indexNames = nearby_venues_clean[ (nearby_venues_clean['Venue_Main_Category'] == 'Other Food') & (nearby_venues_clean['Venue_Category'].isin(list_Food_filter)) ].index
nearby_venues_clean.drop(indexNames , inplace=True)


for idx,cat_m in nearby_venues_clean['Venue_Main_Category'].iteritems():
    cat=nearby_venues_clean.loc[idx,'Venue_Category']
    if cat_m=='Café' or cat_m=='Gym':
        nearby_venues_clean.loc[idx,'Final_Category']=cat_m
    else:
        nearby_venues_clean.loc[idx,'Final_Category']=cat
    
nearby_venues_clean.head()
#nearby_venues_clean[nearby_venues_clean['Area']=='Raffles Place Area'].groupby(['Venue_Main_Category'])['Venue_Category'].value_counts(normalize=False)

,Area,Area_Latitude,Area_Longitude,Venue_Main_Category,Venue_ID,Venue,Venue_Latitude,Venue_Longitude,Venue_Category,Final_Category
0,Raffles Place Area,1.283969,103.85154,Other Food,53ae48b4498ec970f3cc0455,CITY Hot Pot Shabu shabu,1.284173,103.851585,Hotpot Restaurant,Hotpot Restaurant
1,Raffles Place Area,1.283969,103.85154,Other Food,55fe3da6498e0c5eaa442250,CULINARYON,1.284876,103.850933,Comfort Food Restaurant,Comfort Food Restaurant
2,Raffles Place Area,1.283969,103.85154,Other Food,59538b0ac824ae5235423946,Waa Cow!,1.284284,103.851215,Japanese Restaurant,Japanese Restaurant
3,Raffles Place Area,1.283969,103.85154,Other Food,4bb2edcb2397b713669e37b3,The Salad Shop,1.285523,103.851177,Salad Place,Salad Place
4,Raffles Place Area,1.283969,103.85154,Other Food,4b568e7cf964a520151528e3,The Sandwich Shop,1.284266,103.852673,Sandwich Place,Sandwich Place


In [141]:
venue_freq=pd.crosstab(nearby_venues_clean.Area,nearby_venues_clean.Venue_Main_Category,margins=False)
venue_freq.sort_values('Gym',ascending=False)

Venue_Main_Category,Café,Gym,Other Food
Area,,,
Shenton Way Area,28,22,28
Raffles Place Area,30,17,19
Tanjong Pagar / Anson Road,30,16,25
Orchard Road Area,30,10,30
Suntec City,30,10,29
Marina Bay Area,20,4,25
River Valley,9,4,26


# Conclusion I

There are most Gyms in Shenton Way Area. So, relatively we can say people near Shenton Way Area have more healthy awareness.<br>
Also, relatively there're less Cafe in this area except Marina Bay Area and River Valley.<br>
So, let's focus on this area.

In [124]:
shenton=nearby_venues_clean[nearby_venues_clean['Area']=='Shenton Way Area']
shenton

,Area,Area_Latitude,Area_Longitude,Venue_Main_Category,Venue_ID,Venue,Venue_Latitude,Venue_Longitude,Venue_Category,Final_Category
107,Shenton Way Area,1.277737,103.847622,Other Food,5bab10b9364d97002cc5e458,Muchachos,1.279072,103.847026,Burrito Place,Burrito Place
108,Shenton Way Area,1.277737,103.847622,Other Food,50248ddde4b0bb3db3a6ff8a,Pepper Bowl,1.279371,103.846710,Asian Restaurant,Asian Restaurant
109,Shenton Way Area,1.277737,103.847622,Other Food,5887008018dc5375de1b11e0,Kuro Maguro,1.276699,103.845951,Japanese Restaurant,Japanese Restaurant
110,Shenton Way Area,1.277737,103.847622,Other Food,59410e436f706a795d106c76,Ippudo (一風堂),1.277070,103.845790,Ramen Restaurant,Ramen Restaurant
111,Shenton Way Area,1.277737,103.847622,Other Food,591fc82fe96d0c63d9997b98,Venue By Sebastian,1.276363,103.848251,Restaurant,Restaurant
113,Shenton Way Area,1.277737,103.847622,Other Food,59435ff7bd400978c6a8e5ce,Pura Brasa,1.276937,103.846817,Spanish Restaurant,Spanish Restaurant
114,Shenton Way Area,1.277737,103.847622,Other Food,55a49050498e801fbf03be73,Park Bench Deli,1.279872,103.847287,Deli / Bodega,Deli / Bodega
115,Shenton Way Area,1.277737,103.847622,Other Food,5881fc4924f6110c4752d2ef,Imakatsu,1.276605,103.845823,Japanese Restaurant,Japanese Restaurant
116,Shenton Way Area,1.277737,103.847622,Other Food,5832ed3d6b0cdb3ef0437a72,Guzman y Gomez,1.277070,103.845790,Mexican Restaurant,Mexican Restaurant
117,Shenton Way Area,1.277737,103.847622,Other Food,5a25322142d8c2528e5b178a,Myo Restobar,1.278826,103.848354,Diner,Diner


In [150]:
geo = Nominatim(user_agent='My-IBMNotebook')
address = 'Singapore'
location = geo.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_shenton = folium.Map(location=[latitude, longitude], tiles="Openstreetmap", zoom_start=11)

# set color scheme for the clusters
x = np.arange(3)
ys = [i+x+(i*x)**2 for i in range(3)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
cluster=0
for lat, lon, poi, cat in zip(shenton['Venue_Latitude'], shenton['Venue_Longitude'], shenton['Venue'], shenton['Venue_Main_Category']):
    if cat=='Gym':
        cluster=0
    elif cat=='Café':
        cluster=1
    else:
        cluster=2
        
    label = folium.Popup(str(cat) + '-' + str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_shenton)
       
map_shenton

In [190]:
shenton_gym=shenton[shenton['Venue_Main_Category']=='Gym']
venue_list=[]

In [191]:
for idx,lat in shenton_gym['Venue_Latitude'].iteritems():
    lng=shenton_gym.loc[idx,'Venue_Longitude']
    name=shenton_gym.loc[idx,'Venue']
    sll=str(lat) + ',' + str(lng)
    params = dict(
      client_id=cid,
      client_secret=csecret,
      v='20180323',
      ll=sll,
      radius=50,
      query='Café'
    )
    resp = requests.get(url=url, params=params).json()["response"]['groups'][0]['items']
    main_category="Café"
    venue_list.append([(name,lat,lng,main_category,v['venue']['id'],v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name']) for v in resp])

In [192]:
for idx,lat in shenton_gym['Venue_Latitude'].iteritems():
    lng=shenton_gym.loc[idx,'Venue_Longitude']
    name=shenton_gym.loc[idx,'Venue']
    sll=str(lat) + ',' + str(lng)
    params = dict(
      client_id=cid,
      client_secret=csecret,
      v='20180323',
      ll=sll,
      radius=50,
      query='food'
    )
    resp = requests.get(url=url, params=params).json()["response"]['groups'][0]['items']
    main_category="Other food"
    venue_list.append([(name,lat,lng,main_category,v['venue']['id'],v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name']) for v in resp])

In [193]:
for idx,lat in shenton_gym['Venue_Latitude'].iteritems():
    lng=shenton_gym.loc[idx,'Venue_Longitude']
    name=shenton_gym.loc[idx,'Venue']
    sll=str(lat) + ',' + str(lng)
    params = dict(
      client_id=cid,
      client_secret=csecret,
      v='20180323',
      ll=sll,
      radius=50,
      query='Gym'
    )
    resp = requests.get(url=url, params=params).json()["response"]['groups'][0]['items']
    main_category="Gym"
    venue_list.append([(name,lat,lng,main_category,v['venue']['id'],v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name']) for v in resp])

In [249]:
nearby_shenton = pd.DataFrame([item for venue_list in venue_list for item in venue_list])
nearby_shenton.columns = ['Venue','Gym_Latitude','Gym_Longitude','Venue_Main_Category','Venue_ID','Sub_Venue','Venue_Latitude','Venue_Longitude','Venue_Category']

nearby_shenton.head()

,Venue,Gym_Latitude,Gym_Longitude,Venue_Main_Category,Venue_ID,Sub_Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Virgin Active,1.277055,103.846435,Café,5a1e3809a0215b152b57dedf,Cedele,1.276936,103.846260,Café
1,Virgin Active,1.277055,103.846435,Café,4e256401a80967678ca3d6f2,Da Pai Dang 大牌档,1.277222,103.846639,Café
2,Virgin Active,1.277055,103.846435,Café,4e23c2ca1495f18f036537c2,Share Tea 歇脚亭,1.277387,103.846283,Café
3,Virgin Active,1.277055,103.846435,Café,4d8987d9401a224b3bd88e18,Coffee & Toast,1.276724,103.846634,Café
4,Fitness First Platinum,1.277843,103.847654,Café,5625d21a498e596424dac931,Joe & Dough,1.277715,103.847619,Café


In [250]:
nearby_shenton.groupby(['Venue_Main_Category'])['Venue_Category'].value_counts(normalize=False)

Venue_Main_Category  Venue_Category                          
Café                 Café                                        47
                     Bar                                          3
                     Coffee Shop                                  1
Gym                  Gym / Fitness Center                        21
                     Gym                                          9
                     Yoga Studio                                  9
                     Residential Building (Apartment / Condo)     3
                     Boxing Gym                                   2
                     Climbing Gym                                 2
                     Cycle Studio                                 1
Other food           Japanese Restaurant                          9
                     Café                                         8
                     Asian Restaurant                             4
                     Bakery                           

In [251]:
indexNames = nearby_shenton[ (nearby_shenton['Venue_Category'] == 'Residential Building (Apartment / Condo)')].index
nearby_shenton.drop(indexNames , inplace=True)
nearby_shenton.groupby(['Venue_Main_Category'])['Venue_Category'].value_counts(normalize=False)

list_Food_filter=['Café']
indexNames = nearby_shenton[ (nearby_shenton['Venue_Main_Category'] == 'Other Food') & (nearby_shenton['Venue_Category'].isin(list_Food_filter)) ].index
nearby_shenton.drop(indexNames , inplace=True)

In [252]:
nearby_shenton

,Venue,Gym_Latitude,Gym_Longitude,Venue_Main_Category,Venue_ID,Sub_Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Virgin Active,1.277055,103.846435,Café,5a1e3809a0215b152b57dedf,Cedele,1.276936,103.846260,Café
1,Virgin Active,1.277055,103.846435,Café,4e256401a80967678ca3d6f2,Da Pai Dang 大牌档,1.277222,103.846639,Café
2,Virgin Active,1.277055,103.846435,Café,4e23c2ca1495f18f036537c2,Share Tea 歇脚亭,1.277387,103.846283,Café
3,Virgin Active,1.277055,103.846435,Café,4d8987d9401a224b3bd88e18,Coffee & Toast,1.276724,103.846634,Café
4,Fitness First Platinum,1.277843,103.847654,Café,5625d21a498e596424dac931,Joe & Dough,1.277715,103.847619,Café
5,Vanda Boxing Club,1.279218,103.849207,Café,5a7bf9cd9de23b294af39f80,News & Gossip,1.279123,103.848990,Café
6,Vanda Boxing Club,1.279218,103.849207,Café,59adf5319de23b724ca96ffb,Coffee Hive,1.278909,103.848993,Café
7,GIC Gym Studio,1.277402,103.847426,Café,5625d21a498e596424dac931,Joe & Dough,1.277715,103.847619,Café
8,GIC Gym Studio,1.277402,103.847426,Café,4b4e6d92f964a5206aed26e3,Starbucks,1.277564,103.847058,Coffee Shop
9,GIC Gym Studio,1.277402,103.847426,Café,4b8358b7f964a520570331e3,The Coffee Bean & Tea Leaf,1.277080,103.847433,Café


In [253]:
shenton_gym_freq=pd.crosstab(nearby_shenton.Venue,nearby_shenton.Venue_Main_Category,margins=False)
shenton_gym_freq.sort_values('Gym',ascending=False)

Venue_Main_Category,Café,Gym,Other food
Venue,,,
Bodytec Studio,3,4,6
Athlete Lab,4,3,6
Uppercut Boxing,2,3,5
STILL,2,3,2
Haus Athletics,1,3,0
GuavaLabs,1,3,0
Hale Yoga Studio,2,2,3
Anytime Fitness Cecil Street,1,2,0
URA Gymnasium @ 4th Floor,1,2,4


In [257]:
shenton_gym[['Venue','Venue_Latitude','Venue_Longitude']]

,Venue,Venue_Latitude,Venue_Longitude
431,Virgin Active,1.277055,103.846435
432,Fitness First Platinum,1.277843,103.847654
433,Vanda Boxing Club,1.279218,103.849207
434,GIC Gym Studio,1.277402,103.847426
435,STILL,1.277296,103.848878
436,GuavaLabs,1.277162,103.848760
437,Haus Athletics,1.277023,103.848485
438,Ritual Gym,1.278240,103.848499
439,Gym @ The Clift,1.279529,103.847373
440,Anytime Fitness,1.276658,103.845944


In [260]:
shenton_gym_freq2=pd.merge(shenton_gym_freq,shenton_gym[['Venue','Venue_Latitude','Venue_Longitude']],on='Venue',how='left')
shenton_gym_freq2

,Venue,Café,Gym,Other food,Venue_Latitude,Venue_Longitude
0,Anytime Fitness,7,1,9,1.276658,103.845944
1,Anytime Fitness Cecil Street,1,2,0,1.279642,103.848224
2,Athlete Lab,4,3,6,1.280222,103.846832
3,Bodytec Studio,3,4,6,1.280082,103.847438
4,Boulder Movement,1,2,2,1.277651,103.848880
5,Fitness Chemistry,1,2,4,1.279504,103.846560
6,Fitness First Platinum,1,2,2,1.277843,103.847654
7,Freedom Yoga,0,2,0,1.279755,103.848431
8,GIC Gym Studio,4,1,2,1.277402,103.847426
9,GuavaLabs,1,3,0,1.277162,103.848760


In [269]:
shenton_gym_freq3=shenton_gym_freq2.drop('Venue', 1)

In [270]:
kclusters = 4
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(shenton_gym_freq3)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:30])
print(len(kmeans.labels_))

[3 2 1 1 2 1 2 2 0 2 1 1 2 0 0 2 2 1 1 0 0 0]
22


In [271]:
shenton_gym_freq2['Cluster_Labels'] = kmeans.labels_
shenton_gym_freq2.sort_values('Cluster_Labels',ascending=False)

,Venue,Café,Gym,Other food,Venue_Latitude,Venue_Longitude,Cluster_Labels
0,Anytime Fitness,7,1,9,1.276658,103.845944,3
1,Anytime Fitness Cecil Street,1,2,0,1.279642,103.848224,2
4,Boulder Movement,1,2,2,1.277651,103.848880,2
16,Sweatbox,1,2,0,1.276672,103.848690,2
6,Fitness First Platinum,1,2,2,1.277843,103.847654,2
7,Freedom Yoga,0,2,0,1.279755,103.848431,2
15,STILL,2,3,2,1.277296,103.848878,2
9,GuavaLabs,1,3,0,1.277162,103.848760,2
12,Haus Athletics,1,3,0,1.277023,103.848485,2
18,Uppercut Boxing,2,3,5,1.280330,103.847431,1


In [273]:
shenton=shenton_gym_freq2
geo = Nominatim(user_agent='My-IBMNotebook')
address = 'Singapore'
location = geo.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_shenton2 = folium.Map(location=[latitude, longitude], tiles="Openstreetmap", zoom_start=11)

# set color scheme for the clusters
x = np.arange(4)
ys = [i+x+(i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(shenton['Venue_Latitude'], shenton['Venue_Longitude'], shenton['Venue'], shenton['Cluster_Labels']):        
    label = folium.Popup(str(poi) + ' cluster:' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_shenton2)
       
map_shenton2

# Conclusion II

Relatively cluster 2 is gym indensitive and less Cafe around area. It is recomended.